In [0]:
# Alternative approach using pymysql directly
import pandas as pd
import pymysql
import os
from datetime import datetime

# Your RDS connection information
database_username = 'admin'
database_password = 'Qlj124213226'
database_host = 'lq2242-gr5069.ccqalx6jsr2n.us-east-1.rds.amazonaws.com'
database_port = 3306
database_name = 'f1_predictions'

# File path to predictions CSV
predictions_path = "artifacts/model_predictions.csv"

# Check if predictions file exists
if not os.path.exists(predictions_path):
    raise FileNotFoundError(f"Predictions file not found at {predictions_path}. Please run the model training step first.")

# Load predictions from CSV
print(f"Loading predictions from {predictions_path}...")
predictions_df = pd.read_csv(predictions_path)
print(f"Loaded {len(predictions_df)} predictions.")
print("Sample predictions:")
print(predictions_df.head())

try:
    # Connect to MySQL
    print("Connecting to database...")
    connection = pymysql.connect(
        host=database_host,
        user=database_username,
        password=database_password,
        port=database_port,
        charset='utf8mb4'
    )
    
    cursor = connection.cursor()
    
    # Create database if it doesn't exist
    print(f"Creating database {database_name} if it doesn't exist...")
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
    connection.commit()
    
    # Use the database
    print(f"Using database {database_name}...")
    cursor.execute(f"USE {database_name}")
    
    # Create tables if they don't exist
    print("Creating tables if they don't exist...")
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS model1_predictions (
        id INT AUTO_INCREMENT PRIMARY KEY,
        race_id INT NOT NULL,
        driver_id INT NOT NULL,
        predicted_time FLOAT NOT NULL,
        actual_time FLOAT NOT NULL,
        prediction_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """)
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS model2_predictions (
        id INT AUTO_INCREMENT PRIMARY KEY,
        race_id INT NOT NULL,
        driver_id INT NOT NULL,
        predicted_time FLOAT NOT NULL,
        actual_time FLOAT NOT NULL,
        prediction_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """)
    
    connection.commit()
    print("Tables created/confirmed.")
    
    # Insert Random Forest predictions
    print(f"Inserting {len(predictions_df)} Random Forest predictions...")
    for _, row in predictions_df.iterrows():
        cursor.execute("""
        INSERT INTO model1_predictions (race_id, driver_id, predicted_time, actual_time)
        VALUES (%s, %s, %s, %s)
        """, (
            int(row['race_id']),
            int(row['driver_id']),
            float(row['rf_predicted_time']),
            float(row['actual_time'])
        ))
    
    connection.commit()
    print("Random Forest predictions stored successfully.")
    
    # Insert Gradient Boosting predictions
    print(f"Inserting {len(predictions_df)} Gradient Boosting predictions...")
    for _, row in predictions_df.iterrows():
        cursor.execute("""
        INSERT INTO model2_predictions (race_id, driver_id, predicted_time, actual_time)
        VALUES (%s, %s, %s, %s)
        """, (
            int(row['race_id']),
            int(row['driver_id']),
            float(row['gb_predicted_time']),
            float(row['actual_time'])
        ))
    
    connection.commit()
    print("Gradient Boosting predictions stored successfully.")
    
    # Verify data was stored
    print("\nVerifying stored data:")
    
    print("\nRandom Forest predictions (model1_predictions):")
    cursor.execute("SELECT * FROM model1_predictions LIMIT 5")
    for row in cursor.fetchall():
        print(row)
    
    print("\nGradient Boosting predictions (model2_predictions):")
    cursor.execute("SELECT * FROM model2_predictions LIMIT 5")
    for row in cursor.fetchall():
        print(row)
    
    # Count stored predictions
    cursor.execute("SELECT COUNT(*) FROM model1_predictions")
    rf_count = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(*) FROM model2_predictions")
    gb_count = cursor.fetchone()[0]
    
    print(f"\nTotal predictions stored:")
    print(f"Random Forest: {rf_count}")
    print(f"Gradient Boosting: {gb_count}")
    
    # Close connection
    cursor.close()
    connection.close()
    
    print("\nStep 3 completed successfully: Model predictions have been stored in the database.")

except Exception as e:
    print(f"Error: {e}")

Loading predictions from artifacts/model_predictions.csv...
Loaded 110349 predictions.
Sample predictions:
   race_id  driver_id  actual_time  rf_predicted_time  gb_predicted_time
0      879         30        90194       84993.904097       84547.371266
1      984          1       121990       96537.938040       98130.614082
2     1064        852        80565       99011.607126       95612.273809
3       87         30        74292       82023.650746       81886.105767
4      354         20       118887       94445.677771       92562.272681
Connecting to database...
Creating database f1_predictions if it doesn't exist...
Using database f1_predictions...
Creating tables if they don't exist...
Tables created/confirmed.
Inserting 110349 Random Forest predictions...
Random Forest predictions stored successfully.
Inserting 110349 Gradient Boosting predictions...
Gradient Boosting predictions stored successfully.

Verifying stored data:

Random Forest predictions (model1_predictions):
(1, 879,